In [1]:
import pandas as pd
import duckdb
import os
import os, json
from uuid import uuid4
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [2]:
conn = duckdb.connect('/srv/data/greek/grela.duckdb', read_only=True)

In [3]:
query = """
WITH subcorpora AS (
    SELECT
        SUBSTR(grela_id, 0, INSTR(grela_id, '_')) AS subcorpus,
        COUNT(DISTINCT grela_id) AS works_N,
        COUNT(DISTINCT sentence_id) AS sentences_N,
        COUNT(*) AS tokens_N
    FROM tokens
    GROUP BY subcorpus
)
SELECT * FROM subcorpora
ORDER BY subcorpus;
"""

# Execute the query and fetch as a pandas DataFrame
subcorpus_stats_df = conn.execute(query).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
subcorpus_stats_df

,subcorpus,works_N,sentences_N,tokens_N
0,cc,7819,11835457,201939293
1,emlap,73,220846,3495212
2,lagt,1957,2703678,35808742
3,noscemus,996,11802783,139401899


In [6]:
markdown_table = subcorpus_stats_df.set_index("subcorpus").applymap("{:,.0f}".format).to_markdown()
print(markdown_table)

| subcorpus   | works_N   | sentences_N   | tokens_N    |
|:------------|:----------|:--------------|:------------|
| cc          | 7,819     | 11,835,457    | 201,939,293 |
| emlap       | 73        | 220,846       | 3,495,212   |
| lagt        | 1,957     | 2,703,678     | 35,808,742  |
| noscemus    | 996       | 11,802,783    | 139,401,899 |


/tmp/ipykernel_2492460/3358712856.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  markdown_table = subcorpus_stats_df.set_index("subcorpus").applymap("{:,.0f}".format).to_markdown()


In [15]:
# SQL Query for extractaing data
query = """
SELECT
    t.sentence_id,
    t.grela_id,
    t.token_text,
    t.lemma,
    t.pos AS pos_tag,
    t.char_start,
    t.char_end,
    s.text AS sentence_text,
    s.position AS sentence_position,
    w.*
FROM
    tokens t
JOIN
    sentences s
ON
    t.sentence_id = s.sentence_id
JOIN
    works w
ON
    s.grela_id = w.grela_id
WHERE
    t.lemma = 'labyrinthus';
"""

In [11]:
# Execute the query and fetch the result as a Pandas DataFrame
result_df = conn.execute(query).fetchdf()

In [12]:
result_df.head(5)

,sentence_id,grela_id,token_text,lemma,pos_tag,char_start,char_end,sentence_text,sentence_position,grela_source,grela_id_1,author,title,not_before,not_after,lagt_tlg_epithet,lagt_genre,lagt_provenience,noscemus_place,noscemus_genre,noscemus_discipline,title_short,emlap_noscemus_id,place_publication,place_geonames,author_viaf,title_viaf,date_random
0,cc_12132_48965,cc_12132,labyrinthorum,labyrinthus,NOUN,41,54,"Est et picturae genus maeander, in morem labyr...",48965,cc,cc_12132,Desiderius Erasmus,Adagia1508\n,1466.0,1536.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1493.0
1,cc_12138_539,cc_12138,labyrinthis,labyrinthus,NOUN,1112,1123,"Scribunt Annales Sinici, in eodem Natali solo ...",539,cc,cc_12138,"Kircher, Athanasius","China illustrata, pars III",1602.0,1680.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1644.0
2,cc_12150_169,cc_12150,labyrinthus,labyrinthus,NOUN,26,37,"Caput 48 Psammetichi opus labyrinthus, domos m...",169,cc,cc_12150,Pomponius Mela,De chorographia,45.0,45.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,45.0
3,cc_12156_1754,cc_12156,labyrinthorum,labyrinthus,NOUN,18,31,"Hoc anfractuum et labyrinthorum occasione, et ...",1754,cc,cc_12156,"Vesalius, Andreas","De humani corporis fabrica, liber primus",1514.0,1564.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1528.0
4,cc_12172_17967,cc_12172,Labyrinthus,labyrinthus,PROPN,62,73,"Ut in AEgypti pyramidibus, maximae enim hae fu...",17967,cc,cc_12172,"Cardanus, Hieronymus",De subtilitate,1501.0,1576.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1517.0
